# Import

In [ ]:
!pip install folium

In [ ]:
# 좌표변환 패키지
!pip install pyproj

In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import shape
from shapely.geometry import Polygon
import geopandas as gpd
import folium
from folium.plugins import MarkerCluster
from geopandas.tools import sjoin
from shapely.geometry import Point
import json
from pyproj import CRS

# Load File

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
bu_final = pd.read_csv('/content/drive/MyDrive/silverline/final_data/bu_final_ver3.csv', encoding='utf-8')
bu_final['OLD_SAFE']

In [ ]:
with open('/content/drive/MyDrive/silverline/final_data/BU_GRID.geojson', encoding='utf-8') as file:
    geo_grid = json.loads(file.read())
    file.close()

# '/content/drive/MyDrive/silverline/bu_silver/gird.geojson

In [ ]:
# GeoDataFrame으로 변환
geo_grid_gdf = gpd.GeoDataFrame.from_features(geo_grid['features'])
geo_grid_gdf.crs = CRS("EPSG:5179")

geo_grid_gdf['left'] = geo_grid_gdf['left'].round(3)
geo_grid_gdf['bottom'] = geo_grid_gdf['bottom'].round(3)
geo_grid_gdf['right'] = geo_grid_gdf['right'].round(3)
geo_grid_gdf['top'] = geo_grid_gdf['top'].round(3)

In [ ]:
bu_final

In [ ]:
gdf = bu_final.merge(geo_grid_gdf[['id', 'geometry']], on='id')
gdf = gdf.drop(columns='geometry_x')
gdf = gdf.rename(columns={'geometry_y': 'geometry'})
gdf = gdf[['geometry'] + [col for col in gdf.columns if col != 'geometry']]

In [ ]:
df = gdf.copy()

In [ ]:
df = gpd.GeoDataFrame(df, geometry='geometry', crs="EPSG:5179")

In [ ]:
df

In [ ]:
df['ACC'] = df['ACC'].apply(lambda x: np.NaN if x == 0 else x)

In [ ]:
# '사고다발지역폴리곤' 열의 값들을 GeoJSON 파싱하여 유효한 지오메트리 객체로 변환
df['grid_polygon'] = df['geometry'].apply(lambda x: shape(x))

# GeoDataFrame으로 변환
area_gdf = gpd.GeoDataFrame(df, geometry='grid_polygon')
area_gdf.crs = CRS("EPSG:5179")

# 겹치는 부분을 찾아 격자에 수치 정보 추가
for idx, row in geo_grid_gdf.iterrows():
    intersected_polygons = area_gdf[area_gdf.intersects(row['geometry'])]

    # 격자 내에 다발 폴리곤이 있을 경우, 해당 수치를 격자에 추가
    if not intersected_polygons.empty:
        sum_value = intersected_polygons['ACC'].sum()  # 다발 폴리곤의 수치 합계
        geo_grid_gdf.loc[idx, 'ACC'] = sum_value

# map
map = folium.Map(location=(37.5655866, 126.9777942), zoom_start=11)

# Choropleth을 사용하여 격자에 수치 정보 표시
folium.Choropleth(
    geo_data=geo_grid_gdf,
    data=area_gdf,
    columns=['id', 'ACC'],
    key_on='feature.properties.id',
    nan_fill_color='black',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.3,
    legend_name='ACC',
    bins=4
).add_to(map)

map

In [ ]:
map.save('map_acc.html')

# Modeling

In [ ]:
df['acc_score'] = df['acc_score'].apply(lambda x: np.NaN if x == 0 else x)
df['injured_score'] = df['injured_score'].apply(lambda x: np.NaN if x == 0 else x)
df_not_null = df[['acc_score', 'injured_score']].dropna()
df_not_null.describe()

In [ ]:
df = df.fillna(0)

> Score Scaling

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# 특성 스케일링을 위한 MinMaxScaler 객체 생성
scaler = MinMaxScaler(feature_range=(0, 10))

# 선택한 특성들을 정규화할 데이터프레임에 적용
selected_features = ['acc_score','acc_ratio','injured_score','injured_ratio']
df[selected_features] = scaler.fit_transform(df[selected_features])

# 정규화가 적용된 mdf 데이터프레임
df[selected_features].value_counts()

In [ ]:
df['danger_score'] = df['acc_score'] + df['acc_ratio'] + df['injured_score'] + df['injured_ratio']

> danger_score 시각화

In [ ]:
df = df.replace(0, np.NaN)

In [ ]:
df.keys()

In [ ]:
# 선택한 변수들과 'injured_score' 간의 상관관계 계산
correlation_matrix = df[['GRD', 'NLH', 'OLD_SAFE', 'FREE_RICE', 'BUS_STATION','HOSPITAL',
                           'MARKET', 'KID_SAFE', 'cop', 'firestation', 'cctv', 'church',
                           'acc_score', 'acc_ratio','injured_score',
                           'injured_ratio', 'danger_score']].corr()

# 'injured_score'와의 상관계수만 출력
correlation_matrix['danger_score']

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# 상관관계 히트맵 그리기
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=.5)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# 선택한 변수들
selected_variables = ['GRD', 'NLH', 'OLD_SAFE', 'FREE_RICE', 'BUS_STATION','HOSPITAL',
                      'MARKET', 'KID_SAFE', 'cop', 'firestation', 'cctv', 'church',
                      'acc_score', 'acc_ratio','injured_score','injured_ratio', 'danger_score']

# 그래프를 한 번에 그릴 수 있는 크기의 그림 생성
fig, axes = plt.subplots(nrows=5, ncols=4, figsize=(16, 12))
fig.subplots_adjust(wspace=0.5, hspace=0.5)  # 그래프 간의 간격 조절

# 변수 간의 산점도 그리기
for i, variable in enumerate(selected_variables):
    row, col = divmod(i, 4)  # 그래프 위치 계산
    ax = axes[row, col]  # 해당 위치의 축 선택
    ax.scatter(df[variable], df['danger_score'], alpha=0.5)
    ax.set_xlabel(variable)
    ax.set_ylabel('danger_score')
    ax.set_title(f'{variable} and danger_score')

# 그래프 출력
plt.tight_layout()
plt.show()

In [ ]:
df['danger_score'] = df['danger_score'].apply(lambda x: np.NaN if x == 0 else x)

In [ ]:
# GeoDataFrame으로 변환
geo_grid_gdf = gpd.GeoDataFrame.from_features(geo_grid['features'])
geo_grid_gdf.crs = CRS("EPSG:5179")

# '다발지역폴리곤' 열의 값들을 GeoJSON 파싱하여 유효한 지오메트리 객체로 변환
df['grid_polygon'] = df['geometry'].apply(lambda x: shape(x))

# GeoDataFrame으로 변환
gdf = gpd.GeoDataFrame(df, geometry='grid_polygon')
gdf.crs = CRS("EPSG:5179")

# 겹치는 부분을 찾아 격자에 수치 정보 추가
for idx, row in geo_grid_gdf.iterrows():
    intersected_polygons = gdf[gdf.intersects(row['geometry'])]

    # 격자 내에 다발 폴리곤이 있을 경우, 해당 수치를 격자에 추가
    if not intersected_polygons.empty:
        sum_value = intersected_polygons['danger_score'].sum()  # 다발 폴리곤의 수치 합계
        geo_grid_gdf.loc[idx, 'danger_score'] = sum_value

# map
map = folium.Map(location=(37.5655866, 126.9777942), zoom_start=11)

# Choropleth을 사용하여 격자에 수치 정보 표시
folium.Choropleth(
    geo_data=geo_grid_gdf,
    data=gdf,
    columns=['id', 'danger_score'],
    key_on='feature.properties.id',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.3,
    legend_name='danger score',
    bins=4
).add_to(map)

map

In [ ]:
map.save('map_danger_score.html')

# Linear Regression

In [ ]:
df3 = df[['geometry','id','GRD','NLH','OLD_SAFE','FREE_RICE','BUS_STATION','HOSPITAL','MARKET','KID_SAFE','cop', 'firestation', 'cctv', 'church','danger_score']].copy()

> 각 변수 위치 시각화

In [ ]:
df3 = df3.fillna(0)

In [ ]:
df3['cnt'] = df3['GRD'] + df3['NLH'] + df3['OLD_SAFE'] + df3['FREE_RICE'] + df3['BUS_STATION'] + df3['HOSPITAL'] + df3['MARKET'] + df3['KID_SAFE'] + df3['cop'] + df3['firestation'] + df3['cctv'] + df3['church']

In [ ]:
from sklearn.preprocessing import StandardScaler

df3 = df3.fillna(0)

# 특성 스케일링을 위한 StandardScaler 객체 생성
scaler = StandardScaler()

# 선택한 특성들을 정규화할 데이터프레임에 적용
selected_features = ['GRD','NLH','OLD_SAFE','FREE_RICE','BUS_STATION','HOSPITAL','MARKET','KID_SAFE','cop', 'firestation', 'cctv', 'church']
df3[selected_features] = scaler.fit_transform(df3[selected_features])

# 정규화가 적용된 mdf 데이터프레임
df3

In [ ]:
# 상관관계 히트맵 그리기
plt.figure(figsize=(10, 8))
sns.heatmap(df3.corr(), annot=True, cmap='coolwarm', linewidths=.5)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

# 특성 스케일링을 위한 StandardScaler 객체 생성
scaler = StandardScaler()

# 선택한 특성들을 정규화할 데이터프레임에 적용
X_normal = df3[['GRD','NLH','OLD_SAFE','FREE_RICE','BUS_STATION','HOSPITAL','MARKET','KID_SAFE','cop', 'cctv', 'church']]
X_normalized = scaler.fit_transform(X_normal)

y = df3['danger_score']

# 데이터를 학습용과 테스트용으로 나눕니다
X_train, X_test, y_train, y_test = train_test_split(X_normalized, y, test_size=0.2, random_state=42)

# 다중 선형 회귀 모델을 생성하고 훈련합니다
model = LinearRegression()
model.fit(X_train, y_train)

# 모델의 성능을 평가합니다
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# 모델의 성능 지표 출력
print("Mean Squared Error (MSE):", mse)
print("R-squared (R2):", r2)

# 학습된 모델의 계수와 절편 출력
print("Coefficients:", model.coef_)   # 각 특성의 계수
print("Intercept:", model.intercept_)  # 절편

In [ ]:
y_pred

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor

param_grid = {
    'n_estimators': np.arange(50, 200, 10),  # 나무의 수
    'max_depth': np.arange(3, 10),  # 나무의 깊이
    'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3],  # 학습률
    'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],  # 데이터 샘플링 비율
    'colsample_bytree': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],  # 특성 샘플링 비율
    'reg_alpha': [0, 0.001, 0.01, 0.1, 1, 10],  # L1 규제
    'reg_lambda': [0, 0.001, 0.01, 0.1, 1, 10]  # L2 규제
}

# 다중 선형 회귀 모델을 생성하고 훈련합니다
model = XGBRegressor()
random_search = RandomizedSearchCV(model,
                                   param_distributions=param_grid,
                                   n_iter=10,
                                   cv=5,
                                   n_jobs=-1,
                                   verbose=1,
                                   random_state=42)

random_search.fit(X_train, y_train)
print("최적 하이퍼파라미터:", random_search.best_params_)

In [ ]:
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error (MSE):", mse)
print("R-squared (R2):", r2)

In [ ]:
import matplotlib.pyplot as plt

# 학습된 모델의 특성 중요도를 얻습니다.
feature_importance = best_model.feature_importances_

# 특성 이름 리스트 생성 (여기에서는 X_train의 열 이름을 사용)
feature_names = list(df3[['GRD','NLH','OLD_SAFE','FREE_RICE','BUS_STATION','HOSPITAL','MARKET','KID_SAFE','cop', 'cctv', 'church']].columns)

# 특성 중요도를 시각화합니다.
plt.figure(figsize=(10, 6))
plt.barh(range(len(feature_importance)), sorted(feature_importance), align='center')
plt.yticks(range(len(feature_importance)), feature_names)
plt.xlabel('Feature Importance')
plt.ylabel('Features')
plt.title('Feature Importance Plot')
plt.show()

In [ ]:
feature_importance

In [ ]:
# 특성 중요도와 특성 이름을 매핑
feature_importance_mapping = list(zip(df3[['GRD','NLH','OLD_SAFE','FREE_RICE','BUS_STATION','HOSPITAL','MARKET','KID_SAFE','cop', 'cctv', 'church']].columns, feature_importance))

# 결과 출력
for feature, importance in feature_importance_mapping:
    print(f"Feature: {feature}, Importance: {importance}")

In [ ]:
# 특성 중요도와 특성 이름을 매핑
feature_importance_mapping = list(zip(df3[['GRD','NLH','OLD_SAFE','FREE_RICE','BUS_STATION','HOSPITAL','MARKET','KID_SAFE','cop', 'cctv', 'church']].columns, feature_importance))

# 가중치 계산 (변수 중요도와 변수 값의 곱)
weighted_features = feature_importance * np.array(df3[['GRD','NLH','OLD_SAFE','FREE_RICE','BUS_STATION','HOSPITAL','MARKET','KID_SAFE','cop', 'cctv', 'church']])

# 위험 지수 계산 (가중치의 합)
risk_score = np.sum(weighted_features, axis=1)

# df에 'risk_index' 컬럼 추가
df3['risk_score'] = risk_score

# 'risk_index'와 'danger_score'를 더하여 'final_score' 컬럼 생성
df3['risk_index'] = df3['risk_score'] + df3['danger_score']

# 결과 확인
df3

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

X = df3[['GRD','NLH','OLD_SAFE','FREE_RICE','BUS_STATION','HOSPITAL','MARKET','KID_SAFE','cop', 'cctv', 'church']]
y = df3[['risk_index']]  # 'risk_index'를 목표 변수로 사용

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

# 모델 평가
from sklearn.metrics import mean_squared_error, r2_score

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")


In [ ]:
y

# 랜덤 추출 확인

In [ ]:
import random

# 기존 데이터프레임(df2)에서 랜덤한 샘플을 선택합니다.
random_sample = df3.sample(n=1, random_state=random.seed())

# 선택한 랜덤 샘플을 출력합니다.
print("Random Sample:")
print(random_sample)

# 선택한 샘플을 모델에 입력하여 위험 지수를 예측합니다.
new_data = random_sample[['GRD','NLH','OLD_SAFE','FREE_RICE','BUS_STATION','HOSPITAL','MARKET','KID_SAFE','cop', 'cctv', 'church']]
new_risk_index = np.sum(feature_importance * new_data.values)
prediction = model.predict(new_data)

print('='*50)
print(f"Risk Index: {random_sample[['risk_index']]}")
print(f"Predicted Risk Index: {prediction[0]}")

# Results

In [ ]:
df_result = df[['geometry','id','GRD','NLH','OLD_SAFE','FREE_RICE','BUS_STATION','HOSPITAL','MARKET','KID_SAFE','cop', 'cctv', 'church','danger_score']].copy()

In [ ]:
df_result = gpd.GeoDataFrame(df_result, geometry='geometry', crs="EPSG:5179")

In [ ]:
df_result = df_result.fillna(1)

In [ ]:
# risk_index 계산
risk_index = np.sum(feature_importance * df_result[['GRD','NLH','OLD_SAFE','FREE_RICE','BUS_STATION','HOSPITAL','MARKET','KID_SAFE','cop', 'cctv', 'church']].values, axis=1)

# df['danger_score'] 더하기
df_result['risk_index'] = risk_index + df_result['danger_score']

# 결과 확인
df_result

In [ ]:
df_result['risk_index'] = df_result['risk_index'].apply(lambda x: np.NaN if x == 0 else x)

# 가중치 더한 사고다발지역 시각화

In [ ]:
# '다발지역폴리곤' 열의 값들을 GeoJSON 파싱하여 유효한 지오메트리 객체로 변환
df_result['grid_polygon'] = df_result['geometry'].apply(lambda x: shape(x))

# GeoDataFrame으로 변환
risk_gdf = gpd.GeoDataFrame(df_result, geometry='grid_polygon')
risk_gdf.crs = CRS("EPSG:5179")

# 겹치는 부분을 찾아 격자에 수치 정보 추가
for idx, row in geo_grid_gdf.iterrows():
    intersected_polygons = risk_gdf[risk_gdf.intersects(row['geometry'])]

    # 격자 내에 다발 폴리곤이 있을 경우, 해당 수치를 격자에 추가
    if not intersected_polygons.empty:
        sum_value = intersected_polygons['risk_index'].sum()  # 다발 폴리곤의 수치 합계
        geo_grid_gdf.loc[idx, 'risk_index'] = sum_value

# map
map = folium.Map(location=(37.5655866, 126.9777942), zoom_start=11)

# Choropleth을 사용하여 격자에 수치 정보 표시
folium.Choropleth(
    geo_data=geo_grid_gdf,
    data=risk_gdf,
    columns=['id', 'risk_index'],
    key_on='feature.properties.id',
    nan_fill_color='black',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.3,
    legend_name='사고위험지역',
    bins=4
).add_to(map)

map

In [ ]:
map.save('map_risk_index.html')

In [ ]:
old = pd.read_csv('/content/drive/MyDrive/silverline/경기도_노인보호구역.csv', encoding='cp949')

In [ ]:
old = old.dropna(subset=['장소유형코드', '소재지도로명주소'])


old1 = old[old['장소유형코드'] == 1]
result = old1[old1['시군구명'].str.contains('부천시')]
result

# **실제 노인보호구역과 비교**

In [ ]:
import folium

m = folium.Map(location=[37.5655866, 126.9777942], zoom_start=11)

folium.Choropleth(
    geo_data=geo_grid_gdf,
    data=risk_gdf,
    columns=['id', 'risk_index'],
    key_on='feature.properties.id',
    nan_fill_color='black',
    fill_color='YlOrRd',
    fill_opacity=0.7,
    line_opacity=0.3,
    legend_name='사고위험지역',
    bins=4
).add_to(m)


for idx, row in result.iterrows():
    folium.Marker(
        location=[row['위도'], row['경도']],  #
        icon=folium.Icon(color='blue', icon='info-sign')
    ).add_to(m)


m.save('choropleth_map_with_markers.html')

# Display the map in a web browser
m


# **추가로 선정할 지역 추천**

1. 부천역 남부사거리 부근
2. 괴안삼거리 부근
3. 역곡 남부역 사거리 부근
4. 중동 사거리 부근
5. 원미사거리 부근
